In [ ]:
import sys
import os
import time
import glob
import pandas as pd
import numpy as np
from math import sqrt
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
print('python version', sys.version_info)
print('tf version', tf.__version__, 'keras version', keras.__version__)

## NASA Statlog Shuttle multivariate

In [2]:
# %%writefile Shuttle-LSTM.py
import sys
import os
import time
import glob
import pandas as pd
import numpy as np
from math import sqrt
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

current_time_millis = lambda: int(round(time.time() * 1000))

"""
Shuttle data set specific data prep

"""
def label_outliers(nasa_df_row):
    if nasa_df_row['class'] == 1 :
        return 0
    else :
        return 1

"""
Shuttle data set specific data prep

"""
def cleanup() :
    colnames =['time', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'class']
    train_df = pd.read_csv("../Stochastic-Methods/data/nasa/shuttle.trn/shuttle.trn",names=colnames,sep=" ")
    test_df = pd.read_csv("../Stochastic-Methods/data/nasa/shuttle.tst",names=colnames,sep=" ")

    # merge train and test
    merged_df = pd.concat([train_df, test_df])
    # print("Unique classes {}".format(np.unique(merged_df['class'].values, return_counts=True)))

    # drop class = 4
    minus4_df = merged_df.loc[merged_df['class'] != 4]
    # print("Frame after dropping 4 \n{}".format(minus4_df))
    # print("Unique classes after dropping 4 {}".format(np.unique(minus4_df['class'].values, return_counts=True)))

    # mark class 1 as inlier and rest as outlier
    is_anomaly_column = minus4_df.apply(lambda row: label_outliers(row), axis=1)
    labelled_df = minus4_df.assign(is_anomaly=is_anomaly_column.values)

    #print("Frame after labelling outliers \n{}".format(labelled_df))
    print("Unique classes after labelling outliers {}".format(np.unique(labelled_df['class'].values, return_counts=True)))
    print("Unique outliers after labelling outliers {}".format(np.unique(labelled_df['is_anomaly'].values, return_counts=True)))

    # sort by time

    sorted_df = labelled_df.sort_values('time')

    #print("Sorted Frame\n{}".format(sorted_df))
    
    return sorted_df

"""
General data read from pandas data frame

"""

def read_data_with_labels(df, timeVariantColumns, labelColumnNum):
#     df = pd.read_csv(file)
    data = df.values.astype('float64')
    tsData = df[timeVariantColumns].values.astype('float64')
    labels = data[:, labelColumnNum].reshape((-1,1))
    tsDataWithLabels = np.hstack((tsData, labels))
    return tsDataWithLabels, data

"""
General data feature scaling

"""
def scale(data):
    scaler = MinMaxScaler(feature_range=(0,1))
    scaler.fit(data)
    return scaler, scaler.transform(data)

"""
General look back data preparation for model input.

Input expected to be a 2D numpy array with last column being label
Returns looked back X and Y; last column in look back Y data returned is label
Only one step ahead prediction setting is expected.

"""

def look_back_and_create_dataset(tsDataWithLabels, look_back = 1):
    lookbackTsDataX = [] 
    lookbackTsDataYAndLabel = []
    for i in range(look_back, len(tsDataWithLabels)):
        a = tsDataWithLabels[i-look_back:i, :-1]
        lookbackTsDataX.append(a)
        lookbackTsDataYAndLabel.append(tsDataWithLabels[i])
    return np.array(lookbackTsDataX), np.array(lookbackTsDataYAndLabel)

"""
General train/test data split 
Input: whole numpy array input dataset (lookedback)

"""

def split_data_set(dataset, split=0.67):
    train_size = int(len(dataset) * split)
    train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
    return train, test

"""
General train/validate data split 

Input: Xtrain, Ytrain numpy array of full train data

"""
def get_train_validation(Xtrain, Ytrain, validation_ratio=0.1):
    validation_size = int(len(Xtrain) * validation_ratio)
    Xtrain, Xvalid = Xtrain[validation_size:], Xtrain[:validation_size]
    Ytrain, Yvalid = Ytrain[validation_size:], Ytrain[:validation_size]
    return Xtrain, Ytrain, Xvalid, Yvalid

"""
This is general baseline model passed to KerasRegressor for model search.

"""

def baseline_model(input_shape, learning_rate):
    def build_model(input_shape=input_shape, n_hidden = 1, n_units = 50, learning_rate = learning_rate):
        model = keras.models.Sequential()
        model.add(keras.layers.InputLayer(input_shape=input_shape))
        for layer in range(n_hidden - 1):
            # return sequence = true for all layers except last layer
            model.add(keras.layers.LSTM(n_units, return_sequences = True, activation = 'relu'))
        model.add(keras.layers.LSTM(n_units, activation = 'relu'))
        model.add(keras.layers.Dense(input_shape[1]))
        optimizer = keras.optimizers.Adam(lr=learning_rate)
        model.compile(loss="mse", optimizer=optimizer)
        return model
    return build_model

"""

General function to get prediction errors/deviations of all features

"""
def get_deviations_mv(model, X, Y):
    deviations = np.absolute(Y - model.predict(X))
    print("Deviations Mins {}, Maxes {}".format(np.amin(deviations, axis=0), np.amax(deviations, axis=0)))    
    return deviations

"""
General function to get is_anomaly (0/1) labels for each record based on record's feature prediction error/deviation.
For a record, if any feature prediction error/deviation is above the threshold percentile, then the record is anomaly. 

"""
def get_records_above_deviation_pctile_mv(model, X, Y, pctile=95):
    deviations = get_deviations_mv(model, X, Y) # n_samples x n_features
    pctileDeviations = np.percentile(deviations, q=pctile, axis=0) # 1 x n_features
    print("Deviations {}th pctiles {}".format(pctile, pctileDeviations ))
    
    deviations_above_threshold = deviations > pctileDeviations # n_samples x n_features
    print("Shape of deviations above threshold matrix {}".format(deviations_above_threshold.shape))

    predicted_labels = np.ndarray((deviations.shape[0], 1)) # n_samples x 1
    predicted_labels_ref = deviations_above_threshold.any(axis = 1, out = predicted_labels, keepdims = True)
    print("Any feature deviation > its {}th pctile deviation based is_anomaly labels {}"
          .format(pctile, np.unique(predicted_labels_ref, return_counts = True)))
    return predicted_labels_ref

"""
General function to get standard binary classification scores i.e. anomaly performance score based on actual versus predicted
is_anomaly labels for records.

"""
def get_classification_metrics(actual, predicted):
    return confusion_matrix(actual, predicted), precision_score(actual, predicted), \
    recall_score(actual, predicted), f1_score(actual, predicted)

############## main for experiments with Shuttle dataset #########################

split = 0.8
look_back = 24
learning_rate = 0.001
n_iter = 1
cv = 3
batch_size=32
early_stop_patience=3
epochs=10
verbosity=0
min_delta=0.0003
pctile = 99.5

param_distribs = {
    "n_hidden": np.arange(1, 5).tolist(), # upto 4 hidden layers    
    "n_units" : np.arange(1, 100).tolist(), # upto 99 units in each hiden layer.
}


sorted_df = cleanup()

timeVariantColumns = ['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8']
labelColumnNum = 10

# read data
tsDataWithLabels, data = read_data_with_labels(sorted_df, timeVariantColumns, labelColumnNum)
# print("Shapes: time variant data array with labels {}, full data {}".format(tsDataWithLabels.shape, data.shape))
# print("Unique outliers in full data array {}".format(np.unique(data[:, -1], return_counts=True)))
# print("Unique outliers in time variant data array with labels {}".format(np.unique(tsDataWithLabels[:, -1], 
#                                                                                    return_counts=True)))

# print(tsDataWithLabels)

# scale data
scaler, tsDataScaled = scale(tsDataWithLabels)

# Get look back data in the 3D array shape (n_samples, n_lookback_steps, n_features)
lookbackX, lookbackY = look_back_and_create_dataset(tsDataScaled, look_back=look_back)
print("Look back data shapes: lookbackX {} lookbackY {}".format(lookbackX.shape, lookbackY.shape))

 # split into train/test
Xtrain_full, Xtest = split_data_set(lookbackX, split=0.8)
Ytrain_full, Ytest = split_data_set(lookbackY[:, :-1], split=0.8)   # exclude labels     

print("Shapes: Xtrain_full {}, Ytrain_full {}, Xtest {}, Ytest {}".format(Xtrain_full.shape, Ytrain_full.shape, 
                                                                          Xtest.shape, Ytest.shape))

# split further full train set into train and validation set
Xtrain, Ytrain, Xvalid, Yvalid = get_train_validation(Xtrain_full, Ytrain_full, validation_ratio=0.1)

print("Shapes: Xtrain {}, Ytrain {}, Xvalid {}, Yvalid {}".format(Xtrain.shape, Ytrain.shape, 
                                                                  Xvalid.shape, Yvalid.shape))


input_shape = (Xtrain.shape[1], Xtrain.shape[2])
regressor = keras.wrappers.scikit_learn.KerasRegressor(build_fn = baseline_model(input_shape=input_shape, 
                                                                         learning_rate=learning_rate))

early_stopping_cb = keras.callbacks.EarlyStopping(patience=early_stop_patience, monitor='val_loss', min_delta=0.0003, 
                                                  restore_best_weights = True)

rnd_search_cv = RandomizedSearchCV(regressor, param_distribs, n_iter = n_iter, cv = cv, verbose = verbosity)

start_millis = current_time_millis()
rnd_search_cv.fit(Xtrain, Ytrain, epochs=epochs, batch_size=batch_size, validation_data=(Xvalid, Yvalid), 
                  callbacks=[early_stopping_cb], verbose=verbosity)


end_millis = current_time_millis()

print("Time to train {}".format(end_millis -start_millis))

model = rnd_search_cv.best_estimator_.model
print("Best parameters {} best score {}:".format(rnd_search_cv.best_params_, -rnd_search_cv.best_score_))

trainMSE = model.evaluate(Xtrain_full, Ytrain_full, verbose = verbosity)
print("Train Score: {0:.5f} MSE {1:.5f} RMSE".format(trainMSE, np.sqrt(trainMSE)))
testMSE = model.evaluate(Xtest, Ytest, verbose = verbosity)
print("Test Score: {0:.5f} MSE {1:.5f} RMSE".format(testMSE, np.sqrt(testMSE)))

modeldir = 'model-shuttle-lstm'
if not os.path.exists(modeldir):
    os.makedirs(modeldir)
modelpath = modeldir + '/' + 'shuttle-lstm.h5'
print("Saving model", modelpath )
model.save(modelpath)        

# get deviations for whole dataset and id records with deviations > pctile threshold and asign an is_anomaly label
predictedLabels = get_records_above_deviation_pctile_mv(model, lookbackX, lookbackY[:, :-1], pctile)

print("Shape of predicted labels {}".format(predictedLabels.shape))

# actual is_anomaly labels in dataset
actualLabels = (data[look_back:, labelColumnNum] != 0.0).astype('int')    
print("Actual is_anomaly labels in data", np.unique(actualLabels, return_counts = True))

# Compare calculated labels and actual labels to find confusion matrix, precision, recall, and F1
conf_matrix, prec, recall, f1 = get_classification_metrics(actualLabels, predictedLabels)
print("Confusion matrix \n{0}\nprecision {1:.5f}, recall {2:.5f}, f1 {3:.5f}".format(conf_matrix, prec, recall, f1))

Unique classes after labelling outliers (array([1, 2, 3, 5, 6, 7]), array([45586,    50,   171,  3267,    10,    13]))
Unique outliers after labelling outliers (array([0, 1]), array([45586,  3511]))
Look back data shapes: lookbackX (49073, 24, 8) lookbackY (49073, 9)
Shapes: Xtrain_full (39258, 24, 8), Ytrain_full (39258, 8), Xtest (9815, 24, 8), Ytest (9815, 8)
Shapes: Xtrain (35333, 24, 8), Ytrain (35333, 8), Xvalid (3925, 24, 8), Yvalid (3925, 8)
369/369 [==============================] - 5s 15ms/step - loss: 0.0013
Time to train 736069
Best parameters {'n_units': 5, 'n_hidden': 1} best score 0.0014293645896638434:
Train Score: 0.00129 MSE 0.03593 RMSE
Test Score: 0.00728 MSE 0.08530 RMSE
Saving model model-shuttle-lstm/shuttle-lstm.h5
Deviations Mins [1.19780280e-06 1.78813934e-07 1.89209647e-07 9.78127504e-08
 4.35774063e-07 2.04993229e-06 1.26384809e-07 2.52984344e-07], Maxes [0.58820552 0.5101077  0.50917581 0.66136318 0.56898034 0.61230803
 0.50706047 0.52676505]
Deviations 99.